In [2]:
from bs4 import BeautifulSoup
import requests

def generate_url(page: int) -> str:
    return f"https://freedom.pl/mieszkania/kolobrzeg/zp/page/{page}/"

def get_soup(url: str) -> BeautifulSoup:
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

In [3]:
flats_urls = set()

for page in range(1, 5):
    url = generate_url(page)
    soup = get_soup(url)

    flats = soup.find_all("div", class_="offert")
    for flat in flats:
        url = flat.find("a")["href"]
        flats_urls.add(url)

In [4]:
import pandas as pd

df1 = pd.DataFrame(flats_urls)
df1

,0
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...
...,...
64,https://freedom.pl/oferta/mieszkanie-na-sprzed...
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...


In [9]:
df1.to_csv('./data/flats_urls.csv', index=False, header=False)

In [5]:
def format_text(text: str) -> str:
    # text = text.lower()
    text = text.strip()
    text = text.replace("\n", "")
    text = text.replace(":", "")
    return text


def fetch_flat(row):
    url = row[0]
    soup = get_soup(url)
    details = soup.find_all("div", class_="detail")

    offer_table = {
        "url": url,
    }

    for detail in details:
        fields = detail.find_all("li")
        for field in fields:
            key = field.find("strong").text
            key = format_text(key)

            value = field.find("small").text
            value = format_text(value)

            offer_table[key] = value

    offer_table["opis"] = soup.find("div", class_="desc-tab").text.strip()

    return offer_table


df2 = df1.apply(fetch_flat, axis=1, result_type="expand")
df2

,url,Rynek,Cena,Powierzchnia,Cena za m2,Liczba pokoi,Piętro,Liczba pięter,Czynsz administracyjny,Typ mieszkania,...,Cena za garaż/Miejsce parkingowe,Komunikacja,Powierzchnia przedpokoju,Alarm,Liczba oddzielnych toalet,Stan mieszkania,Ciepła woda,Rodzaj ogrzewania,Typ okien,Powierzchnia kuchni
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,420 000 zł,26.8 m2,15 672 zł,1,1,7,350,Apartament,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,490 000 zł,36.98 m2,13 250 zł,2,4,4,389,Apartament,...,40 000 zł,Autobus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,900 000 zł,48.47 m2,18 568 zł,2,3,4,1064,Apartament,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,730 739 zł,31.91 m2,22 900 zł,2,3,5,NaN,Apartament,...,60 000 zł,NaN,2.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,1 270 000 zł,97.34 m2,13 047 zł,4,2,4,800,"Apartament ,Jednopoziomowe",...,120 000 zł,Autobus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,370 000 zł,45.3 m2,8 168 zł,3,3,4,530,"Jednopoziomowe ,Rozkładowe",...,NaN,Autobus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,980 000 zł,103.38 m2,9 480 zł,4,6,7,1342,Rozkładowe,...,85 000 zł,Autobus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,830 000 zł,67.7 m2,12 260 zł,3,3,4,800,Rozkładowe,...,100 000 zł,"PKS, Autobus, Kolej",6.4,NaN,NaN,NaN,NaN,NaN,NaN,"11,6 m2"
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,Wtórny,550 000 zł,46.82 m2,11 747 zł,2,2,4,500,Jednopoziomowe,...,80 000 zł,Autobus,3.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df2.to_csv('./data/flats_dirty.csv', index=False)

In [6]:
df3 = df2[["url", "Numer oferty"]]
df3

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,23159/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,31211/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26529/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24123/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24977/3685/OMS
...,...,...
64,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24780/3685/OMS
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28798/3685/OMS
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28566/3685/OMS
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,23471/3685/OMS


In [7]:
import os

def download_photos(row):
    nr_oferty = row["Numer oferty"]
    url = row["url"]

    soup = get_soup(url)

    images = soup.find("div", class_="swiper-container").find_all("img")
    for image in images:
        src = image["src"].split("-")[:-1]
        photo_url = "-".join(src) + ".jpg"

        filename = photo_url.split("/")[-1]

        directory = f"./images/{nr_oferty}"
        os.makedirs(directory, exist_ok=True)

        response = requests.get(photo_url)
        with open(f"{directory}/{filename}", 'wb') as file:
            file.write(response.content)

    return row

df3.apply(download_photos, axis=1)

,url,Numer oferty
0,https://freedom.pl/oferta/mieszkanie-na-sprzed...,23159/3685/OMS
1,https://freedom.pl/oferta/mieszkanie-na-sprzed...,31211/3685/OMS
2,https://freedom.pl/oferta/mieszkanie-na-sprzed...,26529/3685/OMS
3,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24123/3685/OMS
4,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24977/3685/OMS
...,...,...
64,https://freedom.pl/oferta/mieszkanie-na-sprzed...,24780/3685/OMS
65,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28798/3685/OMS
66,https://freedom.pl/oferta/mieszkanie-na-sprzed...,28566/3685/OMS
67,https://freedom.pl/oferta/mieszkanie-na-sprzed...,23471/3685/OMS
